In [68]:
#  Web-Crawler algorithm


#  useful libraries

import time
import requests #to get the html code from the url
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    


def compute_ranks(graph):
    d = 0.8 # damping factor
    numloops = 10
    
    ranks = {}
    npages = len(graph)
    for page in graph:
        ranks[page] = 1.0 / npages
    
    for i in range(0, numloops):
        newranks = {}
        for page in graph:
            newrank = (1 - d) / npages
            for inlink in graph:
                if page in graph[inlink]:
                    newrank = newrank + d * (ranks[inlink]/len(graph[inlink]))
                
            #newrank = newrank + d * ranks[page]/len(graph[page])
            newranks[page] = newrank
        ranks = newranks
    return ranks


def crawl_web(seed, max_C): # returns index, graph of inlinks
    start = time.clock()
    tocrawl = [seed]
    crawled = []
    graph = {}  # <url>, [list of pages it links to]
    index = {}
    while tocrawl:
        if len(crawled) >= max_C:   # stopping the crawler after few pages [DEBUGGING]
            runtime = time.clock() - start 
            return index, graph, crawled, runtime
        
        deb_time = time.clock()
        try:
            page = tocrawl.pop(X.index(next(x for x in X if seed[:seed.find('/',8)] in x))) #keeping on the seed urls (crawling same site)
        except:
            seed = tocrawl.pop()
            page = seed
        print "crawled length: %s ***** tocrawl length: %s ***** Ex. time: %s" % (len(crawled), len(tocrawl), time.clock() - deb_time)
        #page = tocrawl.pop() original
        
        
        if page not in crawled:
            content = get_page(page)
            add_page_to_index(index, page, content)
            outlinks = get_all_links(content)
            outlinks = [u for u in outlinks if not "//t.co" in u and not "twitter" in u and not "tumblr" in u]
            
            ext_outlinks = [ l for l in list(set(outlinks)) if not l.startswith(page[:page.find('/',8)]) ] #just interested in external links
            
            graph[page] = ext_outlinks
            
            
            union(tocrawl, outlinks)
            crawled.append(page)
    runtime = time.clock() - start        
    return index, graph, crawled, runtime


def get_page(url):
    try:
        return requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
                                          AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}).text
    except:
        return ''
    
def get_next_target(page):
    start_link = page.find('<a href="http')
    if start_link == -1: 
        return None, 0
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote + 1)
    url = page[start_quote + 1:end_quote]
    return url, end_quote

    """if url.startswith("http"): # MY MISTAKE: at the first link not beginning with http the
        return url, end_quote     # procedure will return (None,0), and get_all_links will stop
    else:
        return None, 0"""

def get_all_links(page):
    links = []
    while True:
        url, endpos = get_next_target(page)
        if url:
            links.append(url)
            page = page[endpos:]
        else:
            break
    return links


def union(a, b):
    for e in b:
        if e not in a:
            a.append(e)

def add_page_to_index(index, url, content):
    words = content.split()
    for word in words:
        add_to_index(index, word, url)
        
def add_to_index(index, keyword, url):
    if keyword in index:
        index[keyword].append(url)
    else:
        index[keyword] = [url]

def lookup(index, keyword):
    if keyword in index:
        return index[keyword]
    else:
        return None

seed = 'http://www.trentinowine.info/'#'http://winefolly.com/blog/' #

#index, graph = crawl_web(seed)
index, graph, crawled, runtime = crawl_web(seed,40)
#ranks = compute_ranks(graph)


crawled length: 0 ***** tocrawl length: 0 ***** Ex. time: 9.23692714423e-05
crawled length: 1 ***** tocrawl length: 133 ***** Ex. time: 4.61846357211e-05
crawled length: 2 ***** tocrawl length: 142 ***** Ex. time: 4.33337409049e-05
crawled length: 2 ***** tocrawl length: 141 ***** Ex. time: 2.96494108625e-05
crawled length: 3 ***** tocrawl length: 184 ***** Ex. time: 3.87722975574e-05
crawled length: 4 ***** tocrawl length: 183 ***** Ex. time: 4.96057327837e-05
crawled length: 5 ***** tocrawl length: 186 ***** Ex. time: 3.99126438424e-05
crawled length: 6 ***** tocrawl length: 188 ***** Ex. time: 9.00885206647e-05
crawled length: 7 ***** tocrawl length: 196 ***** Ex. time: 6.10093120486e-05
crawled length: 8 ***** tocrawl length: 207 ***** Ex. time: 4.219339462e-05
crawled length: 9 ***** tocrawl length: 210 ***** Ex. time: 4.0482787881e-05
crawled length: 10 ***** tocrawl length: 210 ***** Ex. time: 5.64478687011e-05
crawled length: 10 ***** tocrawl length: 209 ***** Ex. time: 2.90792

In [69]:
print "Runtime: ~ %s seconds." % int(runtime)
print "Total records: ", len(graph)
n = 1
for i in graph:
    print "[%s]" % n, i
    print graph[i]
    n = n + 1
#x = get_page('http://www.amordivino.net')
#links = get_all_links(x)
#print links
#list(set(graph['https://plus.google.com/117967515779403608969']))

Runtime: ~ 60 seconds.
Total records:  40
[1] https://www.reddit.com/user/pinwale?amp=&amp;count=175&amp;after=t3_2m4swd
[u'https://www.transifex.com/projects/p/reddit', u'http://i.imgur.com/eHLx5hp.png', u'http://alienblue.org', u'https://m.reddit.com', u'http://redditgifts.com', u'http://redditmarket.com', u'https://reddit.zendesk.com', u'http://www.reddit.com/r/france/comments/2lzdaq/saviezvous_que_le_fran%C3%A7ais_est_la_3%C3%A8me_plus/']
[2] https://plus.google.com/u/0/112274166717434183985?rel=author
[u'http://www.google.com/policies/privacy/?hl=en-US', u'http://www.google.com/intl/en-US/policies/terms/', u'https://deals.thenextweb.com/giveaways/the-choose-your-own-apple-watch-giveaway?gid=1482138']
[3] http://www.stackcommerce.com
[u'http://blog.stackcommerce.com/category/press/', u'http://blog.stackcommerce.com/category/industry-news/', u'http://blog.stackcommerce.com', u'https://partners.stackcommerce.com/', u'http://thenextweb.com/insider/2013/07/31/stacksocial-signs-strategi

In [56]:
#crawled_50 = crawled
#graph_50 = graph
#runtime_50 = runtime
?crawled.remove

In [65]:
culo = ["azuz","abcdario","marzuc","pilates","karate","azerabczan"]
print culo
culo = [z for z in culo if not "abc" in z and not "zuc" in z]
print culo

['azuz', 'abcdario', 'marzuc', 'pilates', 'karate', 'azerabczan']
['azuz', 'pilates', 'karate']


In [53]:
get_page('https://t.co/sjdTCLCvq0')

u'<head><noscript><META http-equiv="refresh" content="0;URL=https://amp.twimg.com/v/60341cbe-3fa7-4a96-985c-a8a61d29b6b7"></noscript><title>https://amp.twimg.com/v/60341cbe-3fa7-4a96-985c-a8a61d29b6b7</title></head><script>window.opener = null; location.replace("https:\\/\\/amp.twimg.com\\/v\\/60341cbe-3fa7-4a96-985c-a8a61d29b6b7")</script>'